In [83]:
# This entire cell is for game setup and only needs to be run once.

import numpy as np
from random import randint

class Card:
    values = list('A') + list(range(2,11)) + list('J' + 'Q' + 'K')
    suits = ['Spade', 'Diamond', 'Heart', 'Club']

    def __init__(self, value, suit):
        self.value = value
        self.suit = suit

    def get_card(self, print_ascii, visibility, lines):
        if print_ascii == True:
            self.ascii_art(visibility, lines)
        else:
            print(f'{self.value} of {self.suit}s', end = '')
    
    def ascii_art(self, visibility, lines):
        if visibility != 'hide':
            if self.suit == 'Heart':
                lines[0] += f".------."
                lines[1] += f"|{self.value}_  _ |"
                lines[2] += f"|( \/ )|"
                lines[3] += f"| \  / |"
                lines[4] += f"|  \/ {self.value}|"
                lines[5] += f"'------'"
            elif self.suit == 'Diamond':
                lines[0] += f".------."
                lines[1] += f"|{self.value} /\  |"
                lines[2] += f"| /  \ |"
                lines[3] += f"| \  / |"
                lines[4] += f"|  \/ {self.value}|"
                lines[5] += f"'------'"
            elif self.suit == 'Club':
                lines[0] += f".-------."
                lines[1] += f"|{self.value}  _   |"
                lines[2] += f"|  ( )  |"
                lines[3] += f"| (_x_) |"
                lines[4] += f"|   Y  {self.value}|"
                lines[5] += f"'-------'"
            elif self.suit == 'Spade':
                lines[0] += f".-------."
                lines[1] += f"|{self.value}  .   |"
                lines[2] += f"|  / \  |"
                lines[3] += f"| (_,_) |"
                lines[4] += f"|   I  {self.value}|"
                lines[5] += f"'-------'"
        else:
            lines[0] += f".-------."
            lines[1] += f"|░░░░░░░|"
            lines[2] += f"|░░░░░░░|"
            lines[3] += f"|░░░░░░░|"
            lines[4] += f"|░░░░░░░|"
            lines[5] += f"'-------'"
        return lines


global deck
deck = []

class Deck:
    # Creating the deck is handled at initialisation
    def __init__(self):
       for s in range(len(Card.suits)):
        for v in range(len(Card.values)):
            card = Card(Card.values[v], Card.suits[s])
            deck.append(card)

    def print_deck(self):
        print('Cards in deck:')
        for index, card in enumerate(deck):
            if index < len(deck)-1:
                card.get_card(False, 'show', [])
                print('', end = ', ')
            else:
                card.get_card(False, 'show', [])
                print('', end = '\n\n') 

    def print_card_count(self):
        print(f'Remaining cards: {len(deck)}\n')

    def shuffle_deck(self):
        for i in range(0, 3):
            for card in range(0, len(deck)-1):
                random = randint(0, len(deck)-1)
                swap = deck[card]
                deck[card] = deck[random]
                deck[random] = swap

    def deal_cards(self, dealer, player):
        for i in range(0, 2):
            dealer.hand.append(deck.pop(0))
            player.hand.append(deck.pop(0))


class Dealer:
    hand = []

    def show_hand(self, visibility):
        lines = [f'', f'', f'', f'', f'', f'']
        print("Dealer's hand: ")
        for index, card in enumerate(self.hand):
            if index == len(self.hand)-1 and visibility == 'hide':
                card.get_card(True, 'hide', lines)
            else:
                card.get_card(True, 'show', lines)
        for line in lines:
            print(line)

    def get_total(self, checks = True, print_value = True):
        total = 0
        for card in self.hand:
            if card.value == 'A':
                if (total+11) > 21:
                    total += 1
                else:
                    total += 11
            elif card.value == 'J' or card.value == 'Q' or card.value == 'K':
                total += 10
            else:
                total += card.value
        if print_value == True:
            print(f"Dealer's total points: {total}\n")
        if checks == True:
            self.check_bust(total)
            self.check_blackjack(total)
        return total

    def check_bust(self, total):
        if total > 21:
            print('Dealer BUST!')

    def check_blackjack(self, total):
        if total == 21:
            print('Dealer BLACKJACK!')

    def hit(self):
        if self.get_total() < 17:
            print('Dealer draws another card . . .')
            self.hand.append(deck.pop(0))
            self.show_hand('show')
            self.hit()

class Player:
    hand = []
    bet = 0.00
    earnings = 0.00

    def place_bet(self):
        while True:
            try:
                self.bet = float(input("Place your bet! Minimum bet is $5.00."))
            except ValueError:
                print("You must enter a numerical value.")
                continue
            if self.bet > 5.00:
                print(f"You just bet ${self.bet}.")
                break
            else:
                print("Your bet must be a minimum of $5.00.")

    def show_hand(self):
        lines = [f'', f'', f'', f'', f'', f'']
        print("Your hand: ")
        for index, card in enumerate(self.hand):
                card.get_card(True, 'show', lines)
        for line in lines:
            print(line)

    def get_total(self, checks = True, print_value = True):
        total = 0
        for card in self.hand:
            if card.value == 'A':
                if (total+11) > 21:
                    total += 1
                else:
                    total += 11
            elif card.value == 'J' or card.value == 'Q' or card.value == 'K':
                total += 10
            else:
                total += card.value
        if print_value == True:
            print(f'Your total points: {total}\n')
        if checks == True:
            self.check_bust(total, True)
            self.check_blackjack(total, True)
        return total

    def check_bust(self, total, print_outcome = False):
        if total > 21:
            if print_outcome == True:
                print('BUST!')

    def check_blackjack(self, total, print_outcome = False):
        if total == 21:
            if print_outcome == True:
                print('BLACKJACK!')

    def action_prompt(self):
        next_move = input("What would you like to do? Enter 1 to STAND, 2 to HIT, or 3 to DOUBLE DOWN.")
        if (bool(next_move != '1') ^ bool(next_move != '2') ^ bool(next_move != '3')):
            raise Exception("Please enter 1 to STAND, 2 to HIT, or 3 to DOUBLE DOWN.")
        if next_move == '1':
            self.stand()
        elif next_move == '2':
            self.hit()
        elif next_move == '3':
            self.double_down()
        
    def stand(self):
        print("No longer drawing cards.")
        self.show_hand()
        self.get_total()

    def hit(self):
        print("Drawing another card . . .")
        self.hand.append(deck.pop(0))
        self.show_hand()
        if self.get_total() < 21:
            self.action_prompt()

    def double_down(self):
        print("Doubling your bet and drawing one more card . . .")
        self.bet = self.bet * 2
        self.hand.append(deck.pop(0))
        self.show_hand()
        self.get_total()
    
    def endgame(self, dealer_total = None):
        if dealer_total is None:
            dealer_total = 100
        if bool((self.get_total(False, False) > dealer_total) and (self.get_total(False, False) <= 21)) ^ bool((self.get_total(False, False) <= 21) and (dealer_total > 21)):
            self.earnings += self.bet
            print(f"You win! Your current earnings are: ${self.earnings}.")
        elif bool((self.get_total(False, False) < dealer_total) and dealer_total <= 21) ^ bool(self.get_total(False, False) > 21):
            self.earnings -= self.bet
            print(f"You lose. Your current earnings are: ${self.earnings}.")
        elif self.get_total(False, False) == dealer_total:
            print(f"Draw! Your current earnings are: ${self.earnings}.")

In [84]:
# --- Setting up the deck ---
d = Deck()
d.shuffle_deck()
d.print_deck()
d.print_card_count()

Cards in deck:
9 of Spades, 6 of Spades, Q of Hearts, 6 of Hearts, 3 of Clubs, K of Spades, 3 of Hearts, 5 of Diamonds, 4 of Hearts, 7 of Hearts, 6 of Clubs, 10 of Hearts, 6 of Diamonds, 5 of Spades, A of Diamonds, K of Clubs, 9 of Hearts, 5 of Hearts, 3 of Spades, 3 of Diamonds, Q of Spades, 9 of Clubs, 7 of Diamonds, 10 of Diamonds, 10 of Clubs, A of Hearts, A of Clubs, 2 of Spades, 10 of Spades, K of Diamonds, 8 of Spades, J of Spades, 2 of Clubs, 5 of Clubs, J of Hearts, 9 of Diamonds, Q of Clubs, Q of Diamonds, 7 of Clubs, 2 of Diamonds, K of Hearts, J of Diamonds, 8 of Hearts, 4 of Clubs, 7 of Spades, A of Spades, 8 of Diamonds, 8 of Clubs, 4 of Spades, J of Clubs, 4 of Diamonds, 2 of Hearts

Remaining cards: 52



In [85]:
player = Player()
player.place_bet()

You just bet $30.0.


In [86]:
# dealer 
dealer = Dealer()
d.deal_cards(dealer, player)
dealer.show_hand('hide')
d.print_card_count()

Dealer's hand: 
.-------..-------.
|9  .   ||░░░░░░░|
|  / \  ||░░░░░░░|
| (_,_) ||░░░░░░░|
|   I  9||░░░░░░░|
'-------''-------'
Remaining cards: 48



In [87]:
player.show_hand()
player.get_total()

Your hand: 
.-------..------.
|6  .   ||6_  _ |
|  / \  ||( \/ )|
| (_,_) || \  / |
|   I  6||  \/ 6|
'-------''------'
Your total points: 12



12

In [88]:
player.action_prompt()

Doubling your bet and drawing one more card . . .
Your hand: 
.-------..------..-------.
|6  .   ||6_  _ ||3  _   |
|  / \  ||( \/ )||  ( )  |
| (_,_) || \  / || (_x_) |
|   I  6||  \/ 6||   Y  3|
'-------''------''-------'
Your total points: 15



In [89]:
dealer.show_hand('show')
dealer.hit()
player.endgame(dealer.get_total(False, False))

Dealer's hand: 
.-------..------.
|9  .   ||Q_  _ |
|  / \  ||( \/ )|
| (_,_) || \  / |
|   I  9||  \/ Q|
'-------''------'
Dealer's total points: 19

You lose. Your current earnings are: $-60.0.
